In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
pip install protobuf transformers==4.30.2 cpm_kernels torch>=2.0 gradio mdtex2html sentencepiece accelerate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [ ]:
! pip install opencc-python-reimplemented

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 7.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel
import opencc
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm3-6b", trust_remote_code=True).half()
model = model.to(device)
model = model.eval()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil
import os
def copyfile(file_list, folder=''):
  for file in file_list:
    shutil.copyfile(file, os.path.join(folder,os.path.basename(file)))

list_py = ['/content/gdrive/Othercomputers/我的筆記型電腦/google drive/Sign Sentence Dataset/utils.py']

copyfile(list_py)
shutil.copytree('/content/gdrive/Othercomputers/我的筆記型電腦/google drive/Sign Sentence Dataset/2021-12-3-1400',
        '/content/2021-12-3-1400')


'/content/2021-12-3-1400'

In [4]:
from utils import parse_srt, format_srt
import os
import opencc
import numpy as np
cc = opencc.OpenCC('hk2s')
# str_path = r'/content/2021-12-3-1400/2021-12-3-1400.srt'
str_path = r'2021-12-3-1400\2021-12-3-1400.srt'


In [5]:
srt = parse_srt(str_path)

In [6]:
srt[36]

{'index': 37,
 'start_time': '00:08:33,120',
 'end_time': '00:08:37,169',
 'start_seconds': 513.12,
 'end_seconds': 517.169,
 'text': '這個排列上的第1位 案號16646 這'}

In [11]:
context_list = []
filter_text = ['好','那']
for x in srt:
    try:
        _ = x['text']
        switch = True
    except:
        switch = False
    if switch:
        word_time_position = np.round(np.linspace(x['start_seconds'],
                                                    x['end_seconds'],
                                                   len(x['text'])), 2)
        
        l_point = 0
        for sentence in x['text'].split():
            if sentence not in filter_text:
                context_list.append({'sentence':sentence,
                                        'start_seconds':word_time_position[l_point],
                                        'end_seconds':word_time_position[l_point+len(sentence)-1]})
                l_point += len(sentence)+1

In [1]:
'這'.split()

['這']

In [ ]:
word_time_position = np.round(np.linspace(x['start_seconds'],
                                                    x['end_seconds'],
                                                    len(x['text'])), 2)

In [12]:
context_list

[{'sentence': '各位媒體朋友大家好', 'start_seconds': 407.97, 'end_seconds': 408.98},
 {'sentence': '那今天是由我跟羅副', 'start_seconds': 409.23, 'end_seconds': 410.24},
 {'sentence': '一起召開記者會', 'start_seconds': 410.61, 'end_seconds': 412.3},
 {'sentence': '那我們今天0本土', 'start_seconds': 412.87, 'end_seconds': 414.84},
 {'sentence': '病例', 'start_seconds': 414.87, 'end_seconds': 415.04},
 {'sentence': '另外有11例的境外移入跟', 'start_seconds': 415.36, 'end_seconds': 417.18},
 {'sentence': '0死亡病例', 'start_seconds': 417.18, 'end_seconds': 418.23},
 {'sentence': '那我們的11例境外移入個案是4例',
  'start_seconds': 418.65,
  'end_seconds': 421.44},
 {'sentence': '的男性7例女性', 'start_seconds': 421.44, 'end_seconds': 422.74},
 {'sentence': '那年齡介於20多歲到', 'start_seconds': 423.18, 'end_seconds': 425.13},
 {'sentence': '40多歲', 'start_seconds': 425.67, 'end_seconds': 426.4},
 {'sentence': '那分別來自美國', 'start_seconds': 426.88, 'end_seconds': 428.33},
 {'sentence': '土耳其', 'start_seconds': 428.82, 'end_seconds': 429.3},
 {'sentence': '印尼有4例', 'start

In [106]:
import jieba
text = '我來到北京清華大學'
print('預設:', '|'.join(jieba.cut(text, cut_all=False, HMM=True)))

預設: 我來|到|北京|清華大學


In [107]:
filter_words = ['那']

t = cc.convert('的男性7例女性')
splited_sentence = [i for i in jieba.cut(t)]
print([item for item in splited_sentence if not (item.replace('.', '', 1).isnumeric() if '.' in item else item.isnumeric()) and item not in filter_words])

['的', '男性', '例', '女性']


In [123]:
improved_context = []
concated_sentence= ''
end_seconds = -1
start_seconds = -1
for i, x in enumerate(context_list):
    sentence = x['sentence']
    splited_sentence = [i for i in jieba.cut(cc.convert(sentence), cut_all=False, HMM=True)]
    filtered_list = [item for item in splited_sentence if not item.isdigit()]
    # print(filtered_list, x['sentence'])
    if len(filtered_list) < 4:
        concated_sentence += sentence

    else:
        end_seconds = context_list[i-1]['end_seconds']
        improved_context.append({'sentence':concated_sentence,
                                 'start_seconds':start_seconds,
                                 'end_seconds':end_seconds})
        concated_sentence = sentence
        start_seconds = x['start_seconds']

In [132]:
for i, x in enumerate(improved_context):
    if x['sentence']=='':
        improved_context.pop(i)

In [133]:
improved_context

[{'sentence': '各位媒體朋友大家好', 'start_seconds': 408.22, 'end_seconds': 409.23},
 {'sentence': '那今天是由我跟羅副一起召開記者會',
  'start_seconds': 409.48,
  'end_seconds': 412.3},
 {'sentence': '那我們今天0本土病例', 'start_seconds': 412.87, 'end_seconds': 415.04},
 {'sentence': '另外有11例的境外移入跟0死亡病例',
  'start_seconds': 415.36,
  'end_seconds': 418.23},
 {'sentence': '那我們的11例境外移入個案是4例',
  'start_seconds': 418.65,
  'end_seconds': 421.44},
 {'sentence': '的男性7例女性', 'start_seconds': 421.44, 'end_seconds': 422.74},
 {'sentence': '那年齡介於20多歲到40多歲', 'start_seconds': 423.18, 'end_seconds': 426.4},
 {'sentence': '那分別來自美國土耳其印尼有4例還有西班牙義大利跟韓國英國越南',
  'start_seconds': 426.88,
  'end_seconds': 434.46},
 {'sentence': '那入境日介於今年10月24號到12月1號',
  'start_seconds': 435.06,
  'end_seconds': 438.86},
 {'sentence': '那皆持有搭機前3日內檢驗陰性報告那另外我們昨天總共施打的疫苗',
  'start_seconds': 439.3,
  'end_seconds': 449.49},
 {'sentence': '總共施打的疫苗數是16萬5188劑八起',
  'start_seconds': 454.26,
  'end_seconds': 458.94},
 {'sentence': '那所以我們的疫苗接種人口涵蓋率第一劑是78.08',
  'start

In [140]:
sentence = ''
for i, text in enumerate(sentence_list):
  if i>=4: sentence+=text
  if i==8: break
print(sentence)

那我們今天0本土病例另外有11例的境外移入跟0死亡病例那我們的11例境外移入個案是4例


In [141]:
response, history = model.chat(tokenizer, '你现在要作为一个文字转化器，将输入的句子进行简化并断句', history=[])
print(response)
response, _ = model.chat(tokenizer, f'"{sentence}"', history=history)
print(response[response.find('：')+1:])

好的，请输入需要简化和断句的句子，我会帮您进行处理。
"那我们今天0本土病例，另外有11例的境外移入，跟0死亡病例。那我们的11例境外移入个案，是4例。"


In [142]:
x = response[response.find('：')+1:]
if x.find('。')==-1: pass
elif x.find('。')<=len(x)-3:
  period_position = x.find('。')-x.count('，')-3
  print(period_position, sentence[period_position-1:period_position+1])

24 死亡


In [144]:
sentence_length = 0
for i, text in enumerate(sentence_list):
  if i>=4: sentence_length+=len(text)

  if sentence_length>=period_position: break
print(i)

7


In [145]:
sentence = ''
for i, text in enumerate(sentence_list):
  if i>=4: sentence+=text
  if i==7: break
print(sentence)

那我們今天0本土病例另外有11例的境外移入跟0死亡病例


In [263]:
r_pt = 0
l_pt = 1
extracted = []

In [264]:
len(sentence_list)

997

In [ ]:
for count in range(len(sentence_list)):
  sentence = ''
  for i, text in enumerate(sentence_list):
    if i>=l_pt: sentence+=text
    if i==r_pt: break
  print('sentence:',sentence)
  # sentence = cc.convert(sentence)
  _, history = model.chat(tokenizer, '你现在要作为一个文字转化器，将输入的句子进行简化并断句', history=[])
  while True:
    response, _ = model.chat(tokenizer, f'"{sentence}"', history=history)
    if response.count('"')<=4: break
  print('response:',response[response.find('：')+1:])
  x = response[response.find('：')+1:]
  if x.find('。')==-1: pass
  elif x.find('。')<=len(x)-3:
    period_position = x.find('。')-x.count('，')*2+1
    print(period_position, sentence[period_position-1:period_position+1])
    sentence_length = 0
    for i, text in enumerate(sentence_list):
      if i>=l_pt:sentence_length+=len(text)

      if sentence_length>=period_position:
        sentence = ''
        for t, text in enumerate(sentence_list):
          if t>=l_pt: sentence+=text
          if t==i: break
        extracted.append((x[:x.find('。')+1], sentence))
        print(extracted[-1])
        l_pt = i+1
        r_pt = i+1
        break

  r_pt+=1

sentence: 
response: 请提供您想要转换的句子，我会尽力帮您进行简化并断句。
sentence: 各位媒體朋友大家好
response: "各位媒体朋友大家好"。
sentence: 各位媒體朋友大家好那今天是由我跟羅副
response: "各位媒體朋友大家好，那今天是由我跟羅副"
sentence: 各位媒體朋友大家好那今天是由我跟羅副一起召開記者會
response: "各位媒体朋友大家好，今天是由我跟罗副一起召开新闻发布会。"
sentence: 各位媒體朋友大家好那今天是由我跟羅副一起召開記者會那我們今天0本土
response: 各位媒体朋友大家好，今天是由我跟罗副一起召开新闻发布会。我们今天0本土。
26 那我
('各位媒体朋友大家好，今天是由我跟罗副一起召开新闻发布会。', '各位媒體朋友大家好那今天是由我跟羅副一起召開記者會那我們今天0本土')
sentence: 病例另外有11例的境外移入跟
response: "另外有11例的境外移入病例是"
sentence: 病例另外有11例的境外移入跟0死亡病例
response: 境外移入病例共11例，无死亡病例。
sentence: 病例另外有11例的境外移入跟0死亡病例那我們的11例境外移入個案是4例
response: “另外有11例境外移入个案，其中4例没有导致死亡病例。”
sentence: 病例另外有11例的境外移入跟0死亡病例那我們的11例境外移入個案是4例的男性7例女性
response: "另外有11例境外移入病例，其中4例男性，7例女性，0例死亡病例。"
sentence: 病例另外有11例的境外移入跟0死亡病例那我們的11例境外移入個案是4例的男性7例女性那年齡介於20多歲到
response: 境外移入的病例中，11例中有4例男性，7例女性，年龄在20多岁到某一点之间。
sentence: 病例另外有11例的境外移入跟0死亡病例那我們的11例境外移入個案是4例的男性7例女性那年齡介於20多歲到40多歲
response: "另外有11例境外移入病例，没有导致任何死亡病例。我们的11例境外移入个案中，4例是男性，7例是女性，年龄在20多岁到40多岁之间。"
17 亡病
('"另外有11例境外移入病例，没有导致任何死亡病例。', '病例另外有11例的境外移入跟0死亡

In [260]:
print(extracted)

[('"各位媒體朋友大家好，今天是由我跟羅副一起召開記者會。', '各位媒體朋友大家好那今天是由我跟羅副一起召開記者會'), ('\n\n"今天0本土病例，11例境外移入，无死亡病例。', '那我們今天0本土病例另外有11例的境外移入跟'), ('"0死亡病例，我们的11例境外移入个案中，4例是男性，7例是女性。', '0死亡病例那我們的11例境外移入個案是4例的男性7例女性'), ('"年齡在20多岁到40多岁之间，分别来自美国、土耳其、印尼。', '那年齡介於20多歲到40多歲那分別來自美國土耳其'), ('“印尼有4例，西班牙、義大利、韓國、英國、越南都有。', '印尼有4例還有西班牙義大利跟韓國英國越南那入境日介於今年10月24號到'), ('"12月1号，所有人都需在搭机前3天内进行核酸检测呈阴性证明。', '12月1號那皆持有搭機前3日內檢驗陰性報告那另外我們昨天總共'), ('"施打的疫苗总共有16萬5188劑，分八起施打。', '施打的疫苗總共施打的疫苗數是16萬5188劑')]


In [ ]:
sentence_list = []
response_list = []
for l in range(0, 20, 1):
  sentence = ''
  for i, x in enumerate(srt):
    if i>=l:
      try:
        sentence += ' '+x['text']
      except:
        pass
    if i==l+1:
      break
  sentence = cc.convert(sentence)
  sentence_list.append(sentence)
  while True:
    response, history = model.chat(tokenizer, '你现在要作为一个文字转化器，将输入的句子进行断句', history=[])
    response, _ = model.chat(tokenizer, f'"{sentence}"', history=history)
    if decide(response):
      break
  response_list.append(response)
  print(response)
  print()

各位媒体朋友大家好，那今天是由我跟罗副一起召开记者会，那我们今天0本土。

"一起召开记者会，那我们今天0本土病例，另外有11例的境外移入跟。"

病例另外有11例的境外移入，跟0死亡病例。

"0死亡病例，那我们的11例境外移入个案是4例。" 这句话可以断为："0死亡病例，那我们的11例境外移入个案是4例。"

"那我们的11例境外移入个案是4例的男性7例女性，那年龄介于20多岁到"

"的男性7例，女性7例，年龄介于20多岁到40多岁，分别来自美国、土耳其。"

"40多岁，那分别来自美国、土耳其、印尼有4例，还有西班牙、义大利跟。"

"印尼有4例，还有西班牙，义大利跟韩国，英国，越南"

"韩国英国越南，那入境日介于今年十月二十四号到"

"那入境日介于今年10月24号到12月1号，那皆持有搭机前3日内检验。"

"12月1号/那皆/持有/搭机前/3日内/检验/阴性报告"

"阴性报告，那另外"

"那另外，我们昨天总"

"我们昨天总共施打的疫苗"

"施打的疫苗 总共施打的疫苗数是16万5188剂" 断句如下：

- 施打的疫苗：这是第一个词组。
- 总共施打的疫苗数：这是第二个词组。
- 是：这是第三个词组。
- 16万5188剂：这是第四个词组。

总共施打的疫苗数是16万5188剂，八起。

"八起，那所以我们的疫苗接种人口涵盖率第一剂"

好的，断句后的句子如下：

"那所以我们的疫苗接种人口涵盖率第一剂是78.08，那第二剂是58.59。"

是78.08，那第二剂是58.59。

"那另外追加剂部分，就第三剂的部分是"



In [ ]:
response_list[1]

'断句结果如下：\n\n一起召开记者会，那我们今天0本土病例，另外有11例的境外移入跟。'

In [ ]:
'：' in response_list[1]

True

In [ ]:
def find_commas(text):
    comma_positions = [i for i, char in enumerate(text) if char == '。']
    return comma_positions

In [ ]:
for text in response_list:
  comma_positions = find_commas(text)
  print(comma_positions)

  for i in range(0, len(comma_positions)-1, 2):
    print(text[comma_positions[i]+1:comma_positions[i+1]+1])
    print()

[12, 29, 59, 84, 136, 174, 207]
那今天是由我跟罗副一起召开记者会。

那我们的11例境外移入个案是4例的男性，7例女性。

那入境日介于今年10月24号到12月1号，那皆持有搭机前3日内检验阴性报告。

[64, 134]
那另外我们昨天总共施打的疫苗总共施打的疫苗数是16万5188剂，八起那所以我们的疫苗接种人口涵盖率第一剂是78.08，那第二剂是58.59。

[180]
[]
[26, 107, 164, 224]
那这一周我们的定序总共有验出15个Delta的病例，也包括1例是属于这个Delta plus，就是AY.4.2这个病例，是我们这个排列上的第1位，案号16646。

我们累积到现在，国内是总共已经有验出3例的Delta plus，都是境外移入，前面的2例是英国来的，那这1例是德国来的。

[92, 154, 197, 210]
我们累积到现在，国内是总共已经有验出3例的Delta plus，都是境外移入，前面的2例是英国来的，那这1例是德国来的首例。

那接种的疫苗有5位是AZ。

[43, 85, 130, 200]
那这15例当中，就是也有总共1、2、3、4、5、6、7、8、9位是属于突破性的感染。

这个突破性感染的，针对这个delta的部分，那我们其实都有针对所有境外移入的病例会去做定序，就是说有的病毒量比较低的就会没有办法成功的定序。

[]
[52, 144, 194, 240]
那到现在为止，我们都没有验到任何Omicron的病例，那但是我们从高风险国家地区来的民众，确实是已经有，我们前天有1位是奈及利亚返国的1位民众，那昨天则是有4位是从马拉威返国的一家人。

那累积到现在我们总共的Delta总共235例，其中就是192例是境外移入，本周是新增15例。

[]
[45, 52]
以上，谢谢好。

[]
[]
[42, 64, 139, 184]
中视新闻提问南非代表处2名同仁确诊这个事情。

那这部分可以再请外交部这边去了解就是，当事人的症状还有，有没有愿意提供这些相关的疾病资讯。

[]
[81]
[235]
[60, 118, 222]
因为整个流行情形事实上已经渐趋严重，所以我想我们就是先用一个大概的盛行率来推估说，应该是Omicron的机会蛮高的。

[27]
[68, 155, 180, 2

In [ ]:
for x in comma_positions:
  print(response[x-5:x+6], '|' ,sentence[x-5:x+6])

5位是AZ。" | 的疫苗有5位是AZ 那
